<a href="https://colab.research.google.com/github/hhhhpark/jeby_2021/blob/master/%EC%B5%9C%EC%A0%80%EB%A7%A4%EB%AC%BC%EA%B2%80%EC%83%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

In [32]:
naver = pd.read_csv('네이버매물0216.csv')
std_table = pd.read_csv('서울500세대이상수정본.csv')
cont = pd.read_csv('실거래최근6개월.csv')

# 1. 네이버 최저 매물 검색


In [33]:
#매물&평형 컬럼 추가
naver['매물&평형'] = naver['complexNo'].astype(str)+ '/' + naver['area2'].astype(str)
naver['key'] = naver['매물&평형']+'&'+naver['dealOrWarrantPrc'].astype(str)

#전체 컬럼에서 저층 매물 제외(1,2,3 & 저)
naver['floorInfo'] = naver['floorInfo'].replace(',','/')
naver['floorInfo2'] = naver['floorInfo'].str.split('/').str[0]
naver = naver.loc[(naver['floorInfo2']!='저')&(naver['floorInfo2']!='B1')&(naver['floorInfo2']!='1')&(naver['floorInfo2']!='2')&(naver['floorInfo2']!='3'),:]

#단지별별 평형&금액 기준으로 최저값 추출
naver_hotdeal = pd.DataFrame(naver.groupby(['매물&평형']).agg({'dealOrWarrantPrc':['count','min','max']})).reset_index()
naver_hotdeal.columns=['매물&평형','count','min','max']
naver_hotdeal['key'] = naver_hotdeal['매물&평형']+'&'+naver_hotdeal['min'].astype(str)
naver2 = naver.loc[:,['key','articleName','buildingName','direction','floorInfo']]
naver2 = naver2.drop_duplicates(subset=['key'])
naver_hotdeal = naver_hotdeal.merge(naver2, how='left', on='key')
naver_hotdeal = naver_hotdeal[['articleName','매물&평형','count','min','max','buildingName','direction','floorInfo']]
naver_hotdeal.columns = ['단지명','매물&평형','매물수','최저가','최고가','최저가(동)','최저가(향)','최저가(층)']
naver_hotdeal

,단지명,매물&평형,매물수,최저가,최고가,최저가(동),최저가(향),최저가(층)
0,등촌주공10단지,10/37,3,75000,80000,1004동,남동향,15/15
1,등촌주공10단지,10/58,7,92000,105500,1001동,남향,4/15
2,주공2단지,100207/39,12,55000,63000,204동,남향,고/25
3,주공2단지,100207/59,14,77000,83000,202동,남향,중/25
4,건영,100235/114,14,127000,145000,101동,남향,4/18
...,...,...,...,...,...,...,...,...
2595,래미안안암,9874/59,4,92000,102000,102동,동향,고/14
2596,래미안안암,9874/84,5,110000,130000,107동,남향,14/15
2597,성내삼성,993/114,5,155000,175000,202동,남동향,18/24
2598,성내삼성,993/59,14,105000,110000,103동,남향,12/19


#2. 최저 실거래가 추출(최근6개월, 1-2층 제외)

In [34]:
#실거래 정보에 complex_no 추가
cont['도로명주소'] = cont['시군구'].str.split(' ').str[0]+" "+cont['시군구'].str.split(' ').str[1]+" "+cont['도로명']
cont['면적'] = cont['전용면적(㎡)'].astype(int)
cont = cont[['도로명주소','면적','단지명','계약년월','거래금액(만원)','층','건축년도']]
complex_no = std_table.loc[:,['도로명주소','complex_no']]
cont_res = cont.merge(complex_no, on='도로명주소',how='left')

#모니터링 단지 추출(서울)
cont_res.dropna(axis=0, subset=['complex_no'],inplace=True)
cont_res['complex_no'] = cont_res['complex_no'].astype(int)
cont_res['매물&평형'] = cont_res['complex_no'].astype(str)+'/'+cont_res['면적'].astype(str)
cont_res = cont_res[['매물&평형','도로명주소','단지명','면적','계약년월','거래금액(만원)','층','건축년도']]
cont_res.dropna(axis=0, subset=['도로명주소'],inplace=True)
cont_res

,매물&평형,도로명주소,단지명,면적,계약년월,거래금액(만원),층,건축년도
0,28/182,서울특별시 강남구 언주로 110,경남2차,182,202111,"380,000",2,1984
1,113151/84,서울특별시 강남구 삼성로 11,디에이치아너힐즈,84,202111,"312,000",16,2019
2,113151/59,서울특별시 강남구 삼성로 11,디에이치아너힐즈,59,202111,"230,000",6,2019
3,112228/59,서울특별시 강남구 선릉로 8,래미안블레스티지,59,202111,"230,000",7,2019
26,65/84,서울특별시 강남구 선릉로 120,개포우성1,84,202111,"311,500",12,1983
...,...,...,...,...,...,...,...,...
12838,824/50,서울특별시 중랑구 동일로 752,한신아파트(103~109),50,202109,"67,800",12,1997
12839,824/50,서울특별시 중랑구 동일로 752,한신아파트(103~109),50,202109,"68,500",26,1997
12840,824/50,서울특별시 중랑구 동일로 752,한신아파트(103~109),50,202109,"64,500",1,1997
12841,824/50,서울특별시 중랑구 동일로 752,한신아파트(103~109),50,202110,"70,000",18,1997


In [35]:
cont_res2 = cont_res.loc[(cont_res['층']!=1)&(cont_res['층']!=2),:]
cont_res2['거래금액(만원)'] = cont_res2['거래금액(만원)'].str.replace(',','')
cont_res2['거래금액(만원)'] = cont_res2['거래금액(만원)'].astype(int)
cont_res2['key'] = cont_res2['매물&평형']+'&'+cont_res2['거래금액(만원)'].astype(str)



cont_low_price = pd.DataFrame(cont_res2.groupby(['매물&평형']).min('거래금액(만원)')).reset_index()
cont_low_price = cont_low_price[['매물&평형','거래금액(만원)']]
cont_low_price['key'] = cont_low_price['매물&평형']+'&'+cont_low_price['거래금액(만원)'].astype(str)
#cont_low_price.loc[cont_low_price['매물&평형']=='22693/100']

cont_res2 = cont_res2[['key','도로명주소','단지명','면적','계약년월','층','건축년도']]
cont_low_price = cont_low_price.merge(cont_res2, how='left', on='key')
cont_low_price

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

,매물&평형,거래금액(만원),key,도로명주소,단지명,면적,계약년월,층,건축년도
0,100207/39,55500,100207/39&55500,서울특별시 동대문구 한천로 248,주공2,39,202109,23,2001
1,100207/59,77000,100207/59&77000,서울특별시 동대문구 한천로 248,주공2,59,202108,21,2001
2,100207/84,90000,100207/84&90000,서울특별시 동대문구 한천로 248,주공1,84,202110,10,2001
3,100235/114,120000,100235/114&120000,서울특별시 동작구 만양로 26,건영,114,202108,14,1997
4,100235/32,60000,100235/32&60000,서울특별시 동작구 만양로 26,건영,32,202201,6,1997
...,...,...,...,...,...,...,...,...,...
1863,971/59,175000,971/59&175000,서울특별시 강남구 삼성로64길 5,대치현대,59,202111,17,1999
1864,971/85,246000,971/85&246000,서울특별시 강남구 삼성로64길 5,대치현대,85,202109,14,1999
1865,978/84,234500,978/84&234500,서울특별시 강남구 개포로110길 15,우성7,84,202109,5,1987
1866,979/134,279000,979/134&279000,서울특별시 강남구 광평로19길 15,목련타운,134,202108,7,1993


#3. 실거래가 & 네이버매물 비교

In [36]:
naver_hotdeal2 = naver_hotdeal.iloc[:,1:]
result = cont_low_price.merge(naver_hotdeal2, on='매물&평형', how='left')
#result.loc[result['매물&평형']=='22693/100']

result = result[['단지명','면적','계약년월','거래금액(만원)','층','건축년도','최저가','최저가(층)']]
result.dropna(axis=0, subset=['최저가'],inplace=True)
result['최저가'] = result['최저가'].astype(int)
result['실거래-최저가'] = result['최저가'] - result['거래금액(만원)']

In [37]:
#result.loc[std_table['단지명']=='장안힐스테이트',:]

#[결과1] 실거래 대비비 1억이하

In [38]:
discount_sum = result.loc[result['실거래-최저가']<=-10000,:]
discount_sum.sort_values(by='실거래-최저가', ascending=True, inplace=True)
discount_sum

#cont_low_price.loc[cont_low_price['단지명']=='동아']
#std_table.loc[std_table['단지명']=='장안힐스테이트',:]

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,단지명,면적,계약년월,거래금액(만원),층,건축년도,최저가,최저가(층),실거래-최저가
447,용산센트럴파크해링턴스퀘어,114,202109,325000,19,2020,280000,고/32,-45000
820,래미안퍼스티지,59,202110,300000,20,2009,260000,고/29,-40000
761,용산파크타워(24-0),124,202112,330000,36,2009,290000,6/37,-40000
222,트리마제,140,202111,459000,41,2017,420000,중/46,-39000
788,반포자이,132,202201,460000,17,2009,430000,중/29,-30000
380,아크로리버뷰신반포,84,202111,360000,9,2018,338000,15/31,-22000
1362,남서울,48,202108,99000,4,1974,79000,5/5,-20000
1069,타워팰리스1,164,202109,420000,34,2002,400000,중/66,-20000
789,반포자이,165,202112,520000,28,2009,500000,중/29,-20000
50,흑석한강센트레빌Ⅱ,119,202108,210000,11,2012,190000,13/17,-20000


#[결과2] 실거래 대비비 10% 이하

In [39]:
discount_perc = result.loc[result['실거래-최저가']<=0,:]
discount_perc['실거래-최저가(비율)']=(discount_perc['실거래-최저가']/discount_perc['거래금액(만원)']*100).astype(int)
discount_perc = discount_perc.loc[discount_perc['실거래-최저가(비율)']<=-10,:]
discount_perc.sort_values(by='실거래-최저가(비율)', ascending=True, inplace=True)
discount_perc

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,단지명,면적,계약년월,거래금액(만원),층,건축년도,최저가,최저가(층),실거래-최저가,실거래-최저가(비율)
1362,남서울,48,202108,99000,4,1974,79000,5/5,-20000,-20
1092,동아,88,202108,110000,10,1988,94000,고/15,-16000,-14
447,용산센트럴파크해링턴스퀘어,114,202109,325000,19,2020,280000,고/32,-45000,-13
820,래미안퍼스티지,59,202110,300000,20,2009,260000,고/29,-40000,-13
761,용산파크타워(24-0),124,202112,330000,36,2009,290000,6/37,-40000,-12
829,중계그린1단지,49,202110,72000,14,1990,63000,12/15,-9000,-12
1018,대상타운현대아파트,60,202108,84700,20,2001,75000,11/23,-9700,-11
921,은평뉴타운상림마을(롯데캐슬A-1),101,202109,138000,9,2008,124000,9/9,-14000,-10
1186,창동주공4단지,36,202109,60000,6,1991,54000,14/15,-6000,-10
1187,창동주공4단지,41,202111,59500,8,1991,53000,13/15,-6500,-10


## [기타]단지 검색

In [1]:
#cont_low_price.loc[cont_low_price['단지명']=='동아']
std_table.loc[std_table['단지명']=='헬리오시티',:]

NameError: ignored